In [29]:
from summarizer import Summarizer
from summarizer import Summarizer,TransformerSummarizer
from pdfminer.high_level import extract_pages
from pdfminer.high_level import extract_text
import torch
import re
from urllib.request import Request, urlopen 
import requests  
import urllib.request  
from pdfminer.high_level import extract_pages
from pdfminer.high_level import extract_text
from summarizer import TransformerSummarizer

In [30]:
## Change these as needed.

In [31]:
EXAMPLE_URLS = ["https://publicpartnershipdata.azureedge.net/gef/GEFProjectVersions/c1366692-d840-ed11-9db0-000d3a31b2d8_PIF.pdf", "https://publicpartnershipdata.azureedge.net/gef/GEFProjectVersions/ac49cabb-44c2-ec11-983e-000d3a3006f7_PIF.pdf", "https://publicpartnershipdata.azureedge.net/gef/GEFProjectVersions/742b9aaf-55ba-ec11-983f-0022480a35bf_STAPReview.pdf"]
TEXT_DESTINATION = ["text1.pdf", "text2.pdf", "text3.pdf"]

## Converting PDFs Link into Local PDFs

In [32]:
for example_url_idx in range(len(EXAMPLE_URLS)):
    
    ## grabbing url and corresponding filename
    cur_url = EXAMPLE_URLS[example_url_idx]
    filename = TEXT_DESTINATION[example_url_idx ]
    
    # accessing websiteto get PDF link
    # user-agent needed to circumvent website blockers (usually)
    req = urllib.request.Request(cur_url, headers={'User-Agent': 'Mozilla/5.0'})
    r  = requests.get(cur_url)
    
    with open(filename, "wb") as code:
        code.write(r.content)
        

## Extracting Text from PDFs Using PDFMINER

In [33]:
text_raw_1 = extract_text('text1.pdf')
text_raw_2 = extract_text('text2.pdf')
text_raw_3 = extract_text('text3.pdf')

raw_texts = [text_raw_1, text_raw_2, text_raw_3]

## PDFMINER Information

PDFMiner is a text extraction tool for PDf documents.
Some resources to see how to use it are:
 1. https://towardsdatascience.com/how-to-extract-text-from-pdf-245482a96de7
 2. https://pypi.org/project/pdfminer/
It should be noted that this module is no longer actively maintained. 

## Experimental Preprocessing on the effect of model output.

In [49]:
def experimental_preprocess(text):
    
    post_process = []
    preprocess = text.split(".")
    
    # This is a variable that can be changed.
    # It represents the minimum # words a line of text can have.
    # This is a very experimental value : should be investigated further.
    VAR_LEN = 3
    
    for i in range(0,len(preprocess)):
        
        temp_str = re.sub(r'[^$a-zA-Z0-9 ]', '', preprocess[i]) 
        temp_str_len = len(temp_str.split(" "))
        
        ## Trying to weed out junk content - if something is less than
        ## This number of characters it shouldn't be included.
        if temp_str_len >= VAR_LEN:
            post_process.append(preprocess[i] + ".")
        
    ## Since we have broken up the characters in the string into an array of characters
    ## It is necessary to put them back together.
    final_text = "".join(post_process)
    
    return final_text
        
        

## Questions of PDF Preprocessing:

1. Should the \n character be included in the final input string? There are some models where this has shown to be potentially beneficial.
2. What is the optimal length of VAR_LEN? The smaller it is the higher the risk of junk characters to get it, but the larger it is the higher the risk of losing potentially valuable information.
3. Are there other potentially better algorithms (perhaps already implemented) for doing this? 

## Creating the final processed texts:

In [50]:
text_processed_1 = experimental_preprocess(text_raw_1)
text_processed_2 = experimental_preprocess(text_raw_2)
text_processed_3 = experimental_preprocess(text_raw_3)

processed_texts = [text_processed_1, text_processed_2, text_processed_3]

## BERT Summarizer Tests

In [51]:
## Going to contrast between processed / preprocessed input outputs
## To analyze the impact of preprocessing
results_raw_bert = []
results_processed_bert = []

In [52]:
## For each text in the raw text collection
## Run it through the out of box BERT model and record the input.
for text in raw_texts:
    model = Summarizer()
    result = model(text, num_sentences=5) # Var to research: Changing num_sentences
    results_raw_bert.append(result)



Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.pred

In [53]:
## Running processed texts through Bert Summarizer
for text in processed_texts:
    model = Summarizer()
    result = model(text, num_sentences=5) # Var to research: Changing num_sentences
    results_processed_bert.append(result)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.pred

## Some background resources into the BERT Model

1.https://en.wikipedia.org/wiki/BERT_(language_model)

2.https://www.techtarget.com/searchenterpriseai/definition/BERT-language-model


## Comparing and Constrasting Between Processed/Unprocessed Products [BERT]

## Text Document 1

In [54]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_bert[0])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_bert[0])



**** UNPROCESSED TEXT RESULT ******


Yes 

 No

If yes, please describe

B: Enabling Activity Summary
Provide a brief description of the Enabling Activity. The platform will consolidate global  nance sources and mapping of available  nancial sources for
biodiversity, including the development of an impact investment and private sector knowledge and capacity development stream to facilitate the creation of
national impact investment platforms and related project pipelines (for example, see the UNDP SDG Investor Platform). 5.1.3 Global Catalogue of Biodiversity Finance Solutions
expanded and a consolidated global database of biodiversity
 nance sources and mapping of available  nancial sources for
biodiversity made available for users around the world. Information generated by the project will enable countries to complete the CBD Financial Reporting Framework. Minimum project monitoring and reporting requirements as required by the GEF:

 
 
 
 
 
Inception Workshop and Report:  A proj

## Text Document 2

In [55]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_bert[1])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_bert[1])



**** UNPROCESSED TEXT RESULT ******


World

In-kind

Recurrent 
expenditures

100,000.00

Total Project Cost($)

6,300,000.00

Describe how any "Investment Mobilized" was identified

Investment mobilized was defined based on amount of in-kind and grant contribution from the executing 

ministry, other contributing government institutions, civil society, private sector and other stakeholders 

active in sustainable land management and conservation initiatives. The  participation  of  local  producers  in  green  value  chains  is  also  being  promoted  by  several 
development  projects  including  One  Village  One  Project  OVOP  funded  by  JICA,  JANEEMO 
agroforestry  initiative  funded  by  the  Scottish  government  aiming  at  strengthening  the  organizations 
capacity in production and marketing. This will in turn 
result in an improved 
reduction of forest crimes and 
illegal forest activities. Innovation:  Collaborative  Forest  Management  (CFM)  mechanism  is  not  yet  

## Text Document 3

In [56]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_bert[2])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_bert[2])



**** UNPROCESSED TEXT RESULT ******


STAP guidelines for screening GEF projects 

Response 

Part I: Project 
Information 
GEF ID 
Project Title 

11004 
Strengthening the resilience of small farmers through Climate 
Smart Agriculture (PRP-AIC) techniques in Tahoua Region. The 
PIF does not offer references for either barrier 
that suggest these emerged from research or 
the farmers themselves, which could be a 
problem:  

1)  The project asserts, as Barrier #3, that a 
lack of access to low-cost, long-term 
financing. During the 
PPG stage, the project will need to 
clearly articulate what a learning 
M&E system looks like – what 

9 

 
Is there a recognition of what adaptations may be required 
during project implementation to respond to changing 
conditions in pursuit of the targeted outcomes? During the PPG 
stage, the project should determine this during 
its gender assessment. How these risks play out will, in large 
part, depend on how the project is 
implemented and the ext

## GPT-2 Model Tests (Unprocessed/Vs Processed)

In [58]:
results_raw_gpt2 = []
results_processed_gpt2 = []

In [59]:


for text in raw_texts:
    GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
    result = ''.join(GPT2_model(text, min_length=20, max_length=100))
    results_raw_gpt2.append(result)



In [60]:


for text in processed_texts:
    GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
    result = ''.join(GPT2_model(text, min_length=20, max_length=100))
    results_processed_gpt2.append(result)



## Document 1

In [61]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_gpt2[0])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_gpt2[0])



**** UNPROCESSED TEXT RESULT ******


Enabling Activity Summary (max. 1.2.1 Nature positive and harmful subsidies and incentives to
biodiversity are assessed. The GPMTSU will hire consultants and service providers as required to
execute the work. Future implementation of  nance plans can be covered through STAR allocations. Project outputs, activities, and budget. Finalize the TOR of the Project Board. The evaluation will be ‘independent, impartial and rigorous’.


**** PROCESSED TEXT RESULT ******


Enabling Activity Summary (max. Project monitoring and evaluation. Project outputs, activities, and budget. Finalize the TOR of the Project Board. The independent TE will take place upon completion of all
major project outputs and activities. Check this box if ESS screening is not required per Agency's regulations

 
.


## Document 2

In [62]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_gpt2[1])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_gpt2[1])



**** UNPROCESSED TEXT RESULT ******


Project Justification

1a. It predominantly 
has rocky soils and sandy clay soils. Approximately  2%  of  Malawi?s  cropland  is  irrigated. Nineteen percent of the country?s total land area is 
protected. The geographic proximity of ? Hence 
making the project aligned with BD2.7. benefit and contribute to ecosystem degradation of 
the Bua River catchment ecosystem. Further procurement of PPE for project participants 
and staff. Describe possible coordination with other relevant GEF-
financed projects and other initiatives. The Environment Management Act (No. The plan has prioritized a number of areas that this concept note seeks to address.


**** PROCESSED TEXT RESULT ******


Project Justification

1a. Approximately  2%  of  Malawi?s  cropland  is  irrigated. especially tobacco, tea, and sugar ? The plan has prioritized a number of areas that 
this project proposal seeks to address. Awareness  is  present  in 
communities thanks to national and 

## Document 3

In [63]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_gpt2[2])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_gpt2[2])



**** UNPROCESSED TEXT RESULT ******


These include: 

1)  Communities need to be sensitized to better land 

management. 2)  The project states that low knowledge and 
technical capacity is limiting the adoption 
of CSA. doi:10.2458/v24i1.20969). Taken as a whole, yes. However, if any of 
these assumptions are incorrect, the project 
will deliver far fewer benefits. The 
financing activities under component 3 are 
meant to be exclusively focused on women. During the PPG 
stage, the project should determine this during 
its gender assessment. Are the identified risks valid and comprehensive? Are the risks 
specifically for things outside the project’s control? 13 

 
 
 
 
 
There is very little direct discussion of project 
sensitivity to climate change. See STAP 
brief on simple future narratives. Is there adequate recognition of previous projects and the 
learning derived from them? Knowledge 
management. 15 

 
 
 
 
Notes 

STAP advisory 
response 
1.


**** PROCESSED TEXT RESUL

## XLNET Testing

In [64]:

results_raw_xlnet = []
results_processed_xlnet = []


In [65]:
for text in raw_texts:
    model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
    result = ''.join(model(text, min_length=20, max_length=100))
    results_raw_xlnet.append(result)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

In [66]:


for text in processed_texts:
    model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
    result = ''.join(model(text, min_length=20, max_length=100))
    results_processed_xlnet.append(result)



Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

## Document 1

In [67]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_xlnet[0])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_xlnet[0])



**** UNPROCESSED TEXT RESULT ******


Enabling Activity Summary (max. This Global Programme has the following six components and expected outcomes:

Component 1. 1.2 Harmful and positive subsidies and
incentives to biodiversity are identi ed. The GPMTSU will hire consultants and service providers as required to
execute the work. E: Monitoring and Evaluation Plan
Describe the budgeted M&E plan. ( N/A

ANNEX B: PROJECT BUDGET TABLE
Attach the project budget table. Will be submitted with CEO endorsement.


**** PROCESSED TEXT RESULT ******


Enabling Activity Summary (max. The GPMTSU will hire consultants and service providers as required to
execute the work. Future implementation of  nance plans can be covered through STAR allocations. E: Monitoring and Evaluation Plan
Describe the budgeted M&E plan. N/A

ANNEX B: PROJECT BUDGET TABLE
Attach the project budget table. ESS screening is not required for EAs but should be included if its

available.


## Document 2

In [68]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_xlnet[1])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_xlnet[1])



**** UNPROCESSED TEXT RESULT ******


Project Justification

1a. Forest land comprises 36% of the total land area. Private  land  is  composed  of 
freehold land, leasehold and customary estates. The plan has prioritized a number of areas that 
this project proposal seeks to address. The geographic proximity of ? will facilitate the extension of the PES activity to the project districts. A baseline study conducted in 2021 revealed that all 
gender groups, men, women, girls? benefit and contribute to ecosystem degradation of 
the Bua River catchment ecosystem. Additional efforts will also be made to enhance capacity and awareness amongst women ? Malawi  stands  to  gain  if  women  are  more 
involved in the entire agricultural value chain. The Environment Management Act (No.


**** PROCESSED TEXT RESULT ******


Project Justification

1a. Project Description 

PART II:  PROJECT JUSTIFICATION 
1a. At 
Independence in 1964, ownership of the estates passed into the hands of the Malawian e

## Document 3

In [69]:
print ("**** UNPROCESSED TEXT RESULT ******\n\n")


print(results_raw_xlnet[2])


print ("\n\n**** PROCESSED TEXT RESULT ******\n\n")



print(results_processed_xlnet[2])



**** UNPROCESSED TEXT RESULT ******


These include: 

1)  Communities need to be sensitized to better land 

management. A description of the products and services which are expected 
to result from the project. 2)  The project states that low knowledge and 
technical capacity is limiting the adoption 
of CSA. In this sense, the 
PIF is right: the farmers perceive CSA as 
risky. Taken as a whole, yes. Is the baseline identified clearly? What activities will be implemented to increase the project’s 
resilience to climate change? Do gender considerations hinder full participation of an 
important stakeholder group (or groups)? If so, how will these 
obstacles be addressed? During the PPG 
stage, the project should determine this during 
its gender assessment. It appears the project is aware of a range of 
projects and initiatives relevant to its goals. Is there adequate recognition of previous projects and the 
learning derived from them? Knowledge 
management. 15 

 
 
 
 
Notes 

STA